# Model writer

This notebook performs analyses described in the 

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

- Writes: Prx dimer-decamer equilibrium model (`Prx_dim-dec.psc`)  
- Writes: ITC Prx dimer-decamer equilibrium model (`ITC_Prx_dim-dec.psc`).
- Writes: RBC PRDX2 Model A with decamerisation (`RBC_PRDX2_modelA_decfull.psc`).
- Writes: Prx dimer-decamer dilution model (`Prx_dim-dec_dil.psc`).
- Writes: HRP PRDX2 model (`HRP_PRDX2.psc`)
- Writes: HRP PRDX2 with decamerisation model (`HRP_PRDX2_decfull.psc`).
- Writes: PRDX2 cycle model (`PRDX2_sys.psc`)
- Writes: PRDX2 cycle with decamerisation model (`PRDX2_sys_decfull.psc`).


**Requirements:**

Python libraries (see **Imports** below)

Prx parameters (filename:`fitted_koff_kon_PRDX1_params.csv`)

Prx decamer reactions (filename: `Prx decamer reactions - no hyperoxidation.txt`)

##### Imports

In [ ]:
import os

import pysces

###### get directory paths

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(prev_dir): os.mkdir(mod_dir) # ensure mod_dir exists


## Write models

### Basic dimer-decamer equilibrium 

#### Prx_dim-dec_equi

In [ ]:
# Define model as a string for writing to .psc file
Prx_dim_dec = """

disso:
    decamers = {5}dimers 
    decamers*koff - dimers**exponent*kon

Prx_total_dimers = 10
dimers = 10
decamers = 0

koff = 1
kon = 1
Kd_app = 1.1

exponent = 5
"""

In [ ]:
# Write model .psc file
with open(os.path.join(mod_dir,"Prx_dim-dec.psc"), 'w') as f:
    f.write(Prx_dim_dec)

In [ ]:
# Load and test model
mod_Prx_dim_dec = pysces.model("Prx_dim-dec.psc", dir=mod_dir)
mod_Prx_dim_dec.koff = mod_Prx_dim_dec.Kd_app*mod_Prx_dim_dec.kon
mod_Prx_dim_dec.doState()
mod_Prx_dim_dec.showModel()

### ITC

#### ITC_Prx_dim-dec.psc

In [ ]:
# Define model as a string for writing to .psc file
ITC_Prx_dim_dec = """
# GlobalUnitDefinitions
UnitVolume: litre, 1.0, -6, 1
# UnitLength: metre, 1.0, 0, 1
UnitSubstance: mole, 1.0, -6, 1
# UnitArea: metre, 1.0, 0, 2
UnitTime: second, 1, 0, 1

# Compartments
Compartment: cell_volume, 1400, 3
    
Disassociation@cell_volume:
    decamers = {5}dimers
    koff*decamers - kon*dimers**exponent

# Initial species
dimers@cell_volume  = 0
decamers@cell_volume  = 0

# Parameters
kon = 1
koff = 1
exponent = 130
Kd_app = 1 # Not needed for sims

# Injection parameters
injection_volume = 1.6
injection_dimer = 0.8
injection_decamer = 6.84
injection_interval = 180
first_inject_ratio = 1

Event: first_injection_prot, _TIME_ > 0.0,0 {
dimers = (dimers*cell_volume + first_inject_ratio*injection_dimer*injection_volume + first_inject_ratio*injection_decamer*injection_volume*5)/(cell_volume+injection_volume)
}

Event: first_injection_vol, _TIME_ > 0.0,0 {
cell_volume = cell_volume + injection_volume
}

Event: injection_prot, cos((_TIME_-1)*2*pi/injection_interval) > 0.99999999999,0 {
decamers = (decamers*cell_volume + injection_decamer*injection_volume)/(cell_volume+injection_volume)
dimers = (dimers*cell_volume + injection_dimer*injection_volume)/(cell_volume+injection_volume)
}

Event: injection_vol, cos((_TIME_-1)*2*pi/injection_interval) > 0.99999999999,0 {
cell_volume = cell_volume + injection_volume 
}
    
"""

In [ ]:
# Write .psc file
with open(os.path.join(mod_dir,"ITC_Prx_dim-dec.psc"), 'w') as f:
    f.write(ITC_Prx_dim_dec)

In [ ]:
# Load and test model
mod_ITC_Prx_dec = pysces.model("ITC_Prx_dim-dec.psc", dir=mod_dir)
mod_ITC_Prx_dec.mode_integrate_all_odes = True
mod_ITC_Prx_dec.SetQuiet()
mod_ITC_Prx_dec.showModel()

### RBC PRDX2

#### RBC_PRDX2_modelA_decfull.psc

In [ ]:
# Assign file names
RBC_PRDX2_modA_name_orig = "RBC_PRDX2_modelA"
dec_rxns_no_hyp = "Prx decamer reactions - no hyperoxidation"
RBC_PRDX2_modA_name_new = "RBC_PRDX2_modelA_decfull"

In [ ]:
# Write model .psc file
written_species = list()
written_parameters = list()

omf =  open(os.path.join(mod_dir,f"{RBC_PRDX2_modA_name_orig}.psc"), "r")
omf_line = next(omf)
drf = open(os.path.join(mod_dir,f"{dec_rxns_no_hyp}.txt"), "r")
drf_line = next(drf)
drf_line = next(drf)

with open(f"{mod_dir}/{RBC_PRDX2_modA_name_new}.psc", "w") as outf:
    
    # Script switches for section ends
    orig_reactions_complete = False
    dec_reactions_complete = False
    orig_parameters_complete = False
    dec_parameters_complete = False
    orig_species_complete = False
    dec_species_complete = False
    orig_references_complete = False
    
    # Write dimer reactions
    while orig_reactions_complete == False:
        print(f"Write dimer reactions")
        if omf_line[0:14] == "### Parameters":
            orig_reactions_complete = True
            outf.write("## Decamer reactions\n")
            omf_line = next(omf)
        elif omf_line[0] == "#":
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer reactions  
    while orig_reactions_complete == True and dec_reactions_complete == False:
        print(f"Write decamer reactions")
        if drf_line[0:14] == "### Parameters":
            dec_reactions_complete = True
            outf.write("### Parameters\n")
            outf.write("## Dimer parameters\n")
            drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer parameters
    while dec_reactions_complete == True and orig_parameters_complete == False:
        print(f"Write dimer parameters")
        split_line = omf_line.split()
        if omf_line[0:11] == "### Species":
            orig_parameters_complete = True
            outf.write("## Decamer parameters\n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            if len(split_line) > 0:
                written_parameters.append(split_line[0])

    # Write decamer parameters       
    while orig_parameters_complete == True and dec_parameters_complete == False:
        print(f"Write decamer parameters")
        split_line = drf_line.split()
        if drf_line[0:11] == "### Species":
            dec_parameters_complete = True
            outf.write("### Species\n")
            outf.write("## Dimer Species\n")
            drf_line = next(drf)
        elif len(split_line) > 0 and split_line[0] != "#":
            # Nest like this so the code won't break when split_line is empty
            if split_line[0] not in written_parameters: 
                written_parameters.append(split_line[0])
                outf.write(drf_line)
                drf_line = next(drf)
            else:
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer species
    while dec_parameters_complete == True and orig_species_complete == False:
        print(f"Write dimer species")
        split_line = omf_line.split()
        if omf_line[0:14] == "### References":
            orig_species_complete = True
            outf.write("## Decamer Species\n")
            omf_line = next(omf)
        elif len(split_line) > 0 and split_line[0] != "#":
            # Lines with # and ending in ":" causes issues
            written_species.append(split_line[0])
            outf.write(f"{split_line[0]} = {split_line[2]} \n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer species
    while orig_species_complete == True and dec_species_complete == False:
        print(f"Write decamer species")
        split_line = drf_line.split()
        if drf_line[0:7] == "### End":
            dec_species_complete = True
            outf.write("### References\n")
            drf_line = next(drf)
        elif len(split_line) > 0:
            if split_line[0] not in written_species:
                written_species.append(split_line[0])
                outf.write(f"{split_line[0]}@Erythrocyte = {split_line[2]} \n")
                drf_line = next(drf)
            else:
                outf.write(drf_line)
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)

    # Write the rest of orig file
    while dec_species_complete == True and orig_references_complete == False:
        print(f"Write references")
        if omf_line[0:7] == "### End":
            orig_references_complete = True
            outf.write("### End\n")
        else:
            outf.write(omf_line)
            omf_line = next(omf)
drf.close()
omf.close()

In [ ]:
# Load and test model
mod_A_dec_nohyp = pysces.model("RBC_PRDX2_modelA_decfull",dir = mod_dir)
mod_A_dec_nohyp.showModel()

### Dilution assay

#### Prx_dim-dec_dil.psc

In [ ]:
# Define model as a string for writing to .psc file
Prx_dim_dec_dil = """
# GlobalUnitDefinitions
UnitVolume: litre, 1.0, -6, 1
# UnitLength: metre, 1.0, 0, 1
UnitSubstance: mole, 1.0, -6, 1
# UnitArea: metre, 1.0, 0, 2
UnitTime: second, 1, 0, 1


# Reactions
Decamer_disassociation:
    SH_SH__SH_SH__SH_SH__SH_SH__SH_SH = {5}SH_SH
    k_Prx_dec_disasso*SH_SH__SH_SH__SH_SH__SH_SH__SH_SH- k_Prx_asso*((SH_SH))**exponent

# Species
SH_SH__SH_SH__SH_SH__SH_SH__SH_SH = 0
SH_SH = 0

# Parameters
k_Prx_dec_disasso = 0
k_Prx_asso = 0
exponent = 5

# Dilution parameters
dilution_factor = 1

"""

In [ ]:
# Write model .psc file
with open(os.path.join(mod_dir,"Prx_dim-dec_dil.psc"), 'w') as f:
    f.write(Prx_dim_dec_dil)

In [ ]:
# Load and test model
mod_Prx_dil_v2 = pysces.model("Prx_dim-dec_dil.psc", dir=mod_dir)
mod_Prx_dil_v2.mode_integrator = 'CVODE'
mod_Prx_dil_v2.showModel()

### HRP assay

#### HRP_PRDX2.psc

In [ ]:
# Define model as a string for writing to .psc file
HRP_PRDX2 = """
### Reactions (Flag for scripting)
HRP_ox:
    HRP + H2O2 > compound_I
    k_HRP*HRP*H2O2

# Oxidation reactions
Prx_R_R_ox:
    SH_SH + H2O2 > SH_SOH
    2*kprx*SH_SH*H2O2

Prx_R_S_ox:
    SH_SS + H2O2 > SOH_SS
    kprx*SH_SS*H2O2

Prx_R_O_ox:
    SH_SOH + H2O2 > SOH_SOH
    kprx*SH_SOH*H2O2

Prx_R_O2_ox:
    SH_SO2H + H2O2 > SOH_SO2H
    kprx*SH_SO2H*H2O2

# Over-oxidation reactions
Prx_O_O_oox:
    SOH_SOH + H2O2 > SOH_SO2H
    2*ksulf*SOH_SOH*H2O2

Prx_O_S_oox:
    SOH_SS + H2O2 > SS_SO2H
    ksulf*SOH_SS*H2O2

Prx_R_O_oox:
    SH_SOH + H2O2 > SH_SO2H
    ksulf*SH_SOH*H2O2

Prx_O_O2_oox:
    SOH_SO2H + H2O2 > SO2H_SO2H
    ksulf*SOH_SO2H*H2O2

# Disulphide bridge formation
Prx_R_O_disulf_form:
    SH_SOH > SH_SS
    kcond*SH_SOH

Prx_O_O_disulf_form:
    SOH_SOH > SOH_SS
    2*kcond*SOH_SOH

Prx_O_S_disulf_form:
    SOH_SS > SS_SS
    kcond*SOH_SS

Prx_O_O2_disulf_form:
    SOH_SO2H > SS_SO2H
    kcond*SOH_SO2H

H2O2_production:
    $pool > H2O2
    0.0

### Parameters (Flag for scripting)
kprx = 100.00 # μM/s
ksulf = 0.012 
kcond = 1.7
kred  = 0.21

k_HRP = 11.00 # μM/s

### Species (Flag for scripting)
HRP = 10.0 # μM
compound_I = 0 # μM

H2O2 = 4.0 # μM

SH_SH = 10.0
SH_SOH = 0.0
SH_SS = 0.0
SH_SO2H = 0.0
SOH_SOH = 0.0
SOH_SS = 0.0
SOH_SO2H = 0.0
SS_SS = 0.0
SS_SO2H = 0.0
SO2H_SO2H = 0.0

### References (Flag for scripting)

### End (Flag for scripting)
"""

In [ ]:
# Write model .psc file
with open(os.path.join(mod_dir,"HRP_PRDX2.psc"), 'w') as f:
    f.write(HRP_PRDX2)

In [ ]:
# Load and test model
mod_HRP_Prx_no_dec = pysces.model("HRP_PRDX2.psc", dir=mod_dir)
mod_HRP_Prx_no_dec.mode_integrator = 'CVODE'

mod_HRP_Prx_no_dec.showModel()

#### HRP_PRDX2_decfull.psc

In [ ]:
# Assign file names
HRP_mod_name_orig = "HRP_PRDX2"
dec_rxns_no_hyp = "Prx decamer reactions - no hyperoxidation"
HRP_mod_name_new = "HRP_PRDX2_decfull"

In [ ]:
# Write model .psc file
written_species = list()
written_parameters = list()

omf =  open(os.path.join(mod_dir,f"{HRP_mod_name_orig}.psc"), "r")
omf_line = next(omf)
drf = open(os.path.join(mod_dir,f"{dec_rxns_no_hyp}.txt"), "r")
drf_line = next(drf)
drf_line = next(drf)

with open(os.path.join(mod_dir,f"{HRP_mod_name_new }.psc"), "w") as outf:
    
    # Script switches for section ends
    orig_reactions_complete = False
    dec_reactions_complete = False
    orig_parameters_complete = False
    dec_parameters_complete = False
    orig_species_complete = False
    dec_species_complete = False
    orig_references_complete = False
    
    # Write dimer reactions
    while orig_reactions_complete == False:
        print(f"Write dimer reactions")
        if omf_line[0:14] == "### Parameters":
            orig_reactions_complete = True
            outf.write("## Decamer reactions\n")
            omf_line = next(omf)
        elif omf_line[0] == "#":
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer reactions  
    while orig_reactions_complete == True and dec_reactions_complete == False:
        print(f"Write decamer reactions")
        if drf_line[0:14] == "### Parameters":
            dec_reactions_complete = True
            outf.write("### Parameters\n")
            outf.write("## Dimer parameters\n")
            drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer parameters
    while dec_reactions_complete == True and orig_parameters_complete == False:
        print(f"Write dimer parameters")
        split_line = omf_line.split()
        if omf_line[0:11] == "### Species":
            orig_parameters_complete = True
            outf.write("## Decamer parameters\n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            if len(split_line) > 0:
                written_parameters.append(split_line[0])

    # Write decamer parameters       
    while orig_parameters_complete == True and dec_parameters_complete == False:
        print(f"Write decamer parameters")
        split_line = drf_line.split()
        if drf_line[0:11] == "### Species":
            dec_parameters_complete = True
            outf.write("### Species\n")
            outf.write("## Dimer Species\n")
            drf_line = next(drf)
        elif len(split_line) > 0 and split_line[0] != "#":
            # Nest like this so the code won't break when split_line is empty
            if split_line[0] not in written_parameters: 
                written_parameters.append(split_line[0])
                outf.write(drf_line)
                drf_line = next(drf)
            else:
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer species
    while dec_parameters_complete == True and orig_species_complete == False:
        print(f"Write dimer species")
        split_line = omf_line.split()
        if omf_line[0:14] == "### References":
            orig_species_complete = True
            outf.write("## Decamer Species\n")
            omf_line = next(omf)
        elif len(split_line) > 0 and split_line[0] != "#":
            # Lines with # and ending in ":" causes issues
            written_species.append(split_line[0])
            outf.write(f"{split_line[0]} = {split_line[2]} \n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer species
    while orig_species_complete == True and dec_species_complete == False:
        print(f"Write decamer species")
        split_line = drf_line.split()
        if drf_line[0:7] == "### End":
            dec_species_complete = True
            outf.write("### References\n")
            drf_line = next(drf)
        elif len(split_line) > 0:
            if split_line[0] not in written_species:
                written_species.append(split_line[0])
                outf.write(f"{split_line[0]} = {split_line[2]} \n")
                drf_line = next(drf)
            else:
                outf.write(drf_line)
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)

    # Write the rest of orig file
    while dec_species_complete == True and orig_references_complete == False:
        print(f"Write references")
        if omf_line[0:7] == "### End":
            orig_references_complete = True
            outf.write("### End\n")
        else:
            outf.write(omf_line)
            omf_line = next(omf)
drf.close()
omf.close()

In [ ]:
# Load and test model
mod_HRP_Prx_with_dec = pysces.model("HRP_PRDX2_decfull.psc", dir=mod_dir)
mod_HRP_Prx_with_dec.mode_integrator = 'CVODE'
mod_HRP_Prx_with_dec.showModel()

### NADPH assay

#### PRDX2_sys.psc

In [ ]:
# Define model as a string for writing to .psc file
PRDX2_sys = """

Function: MM_irr_2S, E, kcat, s1, km_s1, s2, km_s2 {
(E*kcat)*(s1/(km_s1+s1))*(s2/(km_s2+s2))}

### Reactions (Flag for scripting)

# Oxidation reactions
Prx_R_R_ox:
    SH_SH + H2O2 > SH_SOH
    2*kprx*SH_SH*H2O2

Prx_R_S_ox:
    SH_SS + H2O2 > SOH_SS
    kprx*SH_SS*H2O2

Prx_R_O_ox:
    SH_SOH + H2O2 > SOH_SOH
    kprx*SH_SOH*H2O2

Prx_R_O2_ox:
    SH_SO2H + H2O2 > SOH_SO2H
    kprx*SH_SO2H*H2O2

# Over-oxidation reactions
Prx_O_O_oox:
    SOH_SOH + H2O2 > SOH_SO2H
    2*ksulf*SOH_SOH*H2O2

Prx_O_S_oox:
    SOH_SS + H2O2 > SS_SO2H
    ksulf*SOH_SS*H2O2

Prx_R_O_oox:
    SH_SOH + H2O2 > SH_SO2H
    ksulf*SH_SOH*H2O2

Prx_O_O2_oox:
    SOH_SO2H + H2O2 > SO2H_SO2H
    ksulf*SOH_SO2H*H2O2

# Disulphide bridge formation
Prx_R_O_disulf_form:
    SH_SOH > SH_SS
    kcond*SH_SOH

Prx_O_O_disulf_form:
    SOH_SOH > SOH_SS
    2*kcond*SOH_SOH

Prx_O_S_disulf_form:
    SOH_SS > SS_SS
    kcond*SOH_SS

Prx_O_O2_disulf_form:
    SOH_SO2H > SS_SO2H
    kcond*SOH_SO2H

# Disulphide bridge reduction via Thioredoxin
Prx_R_S_red_TrxSH:
    SH_SS + TrxSH > SH_SH + TrxSOH
    kred*SH_SS*TrxSH

Prx_O_S_red_TrxSH:
    SOH_SS + TrxSH > SH_SOH + TrxSOH
    kred*SOH_SS*TrxSH

Prx_S_S_red_TrxSH:
    SS_SS + TrxSH > SH_SS + TrxSOH
    2*kred*SS_SS*TrxSH

Prx_S_O2_red_TrxSH:
    SS_SO2H + TrxSH > SH_SO2H + TrxSOH
    kred*SS_SO2H*TrxSH

# Trx regeneration
Trx_red1:
    TrxSOH + NADPH > NADP + TrxSH
    MM_irr_2S(TRR,kcat_TRR,TrxSOH,KM_TrxSOH_TRR,NADPH,KM_NADPH_TRR)

H2O2_production:
    $pool > H2O2
    0.0

### Parameters (Flag for scripting)
kprx = 100 # μM/s
ksulf = 0.012 
kcond = 1.7
kred  = 0.21

TRR = 0.14
kcat_TRR = 73
KM_TrxSOH_TRR = 1.83
KM_NADPH_TRR = 6.0

### Species (Flag for scripting)
H2O2 = 4.0 # μM

SH_SH = 10.0
SH_SOH = 0.0
SH_SS = 0.0
SH_SO2H = 0.0
SOH_SOH = 0.0
SOH_SS = 0.0
SOH_SO2H = 0.0
SS_SS = 0.0
SS_SO2H = 0.0
SO2H_SO2H = 0.0

TrxSH = 5.0
TrxSOH = 0.0

NADPH = 5.0
NADP = 0.0

### References (Flag for scripting)

### End (Flag for scripting)
"""

In [ ]:
# Write model .psc file
with open(os.path.join(mod_dir,"PRDX2_sys.psc"), 'w') as f:
    f.write(PRDX2_sys)

In [ ]:
# Load and test model
mod_Prx_cycle_no_dec = pysces.model("PRDX2_sys.psc", dir=mod_dir)
mod_Prx_cycle_no_dec.mode_integrator = 'CVODE'
mod_Prx_cycle_no_dec.showModel()

#### PRDX2_sys_decfull.psc

In [ ]:
# Assign file names
dil_mod_name_orig = "PRDX2_sys"
dec_rxns_no_hyp = "Prx decamer reactions - no hyperoxidation"
dil_mod_name_new = "PRDX2_sys_decfull"

In [ ]:
# Write model .psc file
written_species = list()
written_parameters = list()

omf =  open(os.path.join(mod_dir, f"{dil_mod_name_orig}.psc"), "r")
omf_line = next(omf)
drf = open(os.path.join(mod_dir, f"{dec_rxns_no_hyp}.txt"), "r")
drf_line = next(drf)
drf_line = next(drf)

with open(os.path.join(mod_dir, f"{dil_mod_name_new}.psc"), "w") as outf:
    
    # Script switches for section ends 
    orig_reactions_complete = False
    dec_reactions_complete = False
    orig_parameters_complete = False
    dec_parameters_complete = False
    orig_species_complete = False
    dec_species_complete = False
    orig_references_complete = False
    
    # Write dimer reactions
    while orig_reactions_complete == False:
        print(f"Write dimer reactions")
        if omf_line[0:14] == "### Parameters":
            orig_reactions_complete = True
            outf.write("## Decamer reactions\n")
            omf_line = next(omf)
        elif omf_line[0] == "#":
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer reactions  
    while orig_reactions_complete == True and dec_reactions_complete == False:
        print(f"Write decamer reactions")
        if drf_line[0:14] == "### Parameters":
            dec_reactions_complete = True
            outf.write("### Parameters\n")
            outf.write("## Dimer parameters\n")
            drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer parameters
    while dec_reactions_complete == True and orig_parameters_complete == False:
        print(f"Write dimer parameters")
        split_line = omf_line.split()
        if omf_line[0:11] == "### Species":
            orig_parameters_complete = True
            outf.write("## Decamer parameters\n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            if len(split_line) > 0:
                written_parameters.append(split_line[0])

    # Write decamer parameters       
    while orig_parameters_complete == True and dec_parameters_complete == False:
        print(f"Write decamer parameters")
        split_line = drf_line.split()
        if drf_line[0:11] == "### Species":
            dec_parameters_complete = True
            outf.write("### Species\n")
            outf.write("## Dimer Species\n")
            drf_line = next(drf)
        elif len(split_line) > 0 and split_line[0] != "#":
            # Nest like this so the code won't break when split_line is empty
            if split_line[0] not in written_parameters: 
                written_parameters.append(split_line[0])
                outf.write(drf_line)
                drf_line = next(drf)
            else:
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)
            
    # Write dimer species
    while dec_parameters_complete == True and orig_species_complete == False:
        print(f"Write dimer species")
        split_line = omf_line.split()
        if omf_line[0:14] == "### References":
            orig_species_complete = True
            outf.write("## Decamer Species\n")
            omf_line = next(omf)
        # Lines with # and ending in ":" causes issues
        elif len(split_line) > 0 and split_line[0] != "#": 
            written_species.append(split_line[0])
            outf.write(f"{split_line[0]} = {split_line[2]} \n")
            omf_line = next(omf)
        else:
            outf.write(omf_line)
            omf_line = next(omf)
            
    # Write decamer species
    while orig_species_complete == True and dec_species_complete == False:
        print(f"Write decamer species")
        split_line = drf_line.split()
        if drf_line[0:7] == "### End":
            dec_species_complete = True
            outf.write("### References\n")
            drf_line = next(drf)
        elif len(split_line) > 0:
            if split_line[0] not in written_species:
                written_species.append(split_line[0])
                outf.write(f"{split_line[0]} = {split_line[2]} \n")
                drf_line = next(drf)
            else:
                outf.write(drf_line)
                drf_line = next(drf)
        else:
            outf.write(drf_line)
            drf_line = next(drf)

    # Write the rest of orig file
    while dec_species_complete == True and orig_references_complete == False:
        print(f"Write references")
        if omf_line[0:7] == "### End":
            orig_references_complete = True
            outf.write("### End\n")
        else:
            outf.write(omf_line)
            omf_line = next(omf)
drf.close()
omf.close()

In [ ]:
# Load and test model
mod_Prx_cycle_with_dec = pysces.model("PRDX2_sys_decfull.psc", dir=mod_dir)
mod_Prx_cycle_with_dec.mode_integrator = 'CVODE'
mod_Prx_cycle_with_dec.showModel()